In [44]:
import numpy as np
from IPython.core.debugger import set_trace
import re
import difflib
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests
import time
import random
import pickle
from lxml.html import fromstring
from collections import Counter
from requests.packages.urllib3.util import Retry
from requests.adapters import HTTPAdapter
from requests import Session, exceptions
import sys
import pymongo
from pymongo import MongoClient
from newspaper import Article
import json
from string import digits
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
from nltk.stem.porter import *
from collections import Counter 
from nltk.corpus import wordnet # To get words in dictionary with their parts of speech
from nltk.stem import WordNetLemmatizer # lemmatizes word based on it's parts of speech
from nltk.corpus import stopwords 
from textblob import TextBlob

In [3]:
# data load from muckrack
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/Attempt3_mr_scrape.pickle', 'rb') as handle:
    scrapevar = pickle.load(handle)
    
# load writer df
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/writer_df.pickle', 'rb') as handle:
    writer_df = pickle.load(handle)

In [4]:
# Establish connection with database
client = MongoClient()
mydb = client["testinsightdb"]

In [7]:
currcol = mydb['zachlowe_nba']
y = currcol.find({"name": 'zachlowe_nba'})

In [48]:
# preprocessing functions
def clean_text(txtstr):
    txtstr = re.sub(r'\n\s*\n', '', txtstr) # extra lines
    txtstr = re.sub(r'[^A-Za-z0-9(),!?@\'\`\"\_\n]', ' ', txtstr)
    txtstr = re.sub(r'[^\w\s]','',txtstr) # punctuation
    txtstr = re.sub(r'\w*\d\w*','',txtstr) # words with number plust letters
    remove_digits = str.maketrans('', '', digits)
    txtstr = txtstr.translate(remove_digits)
    txtstr = re.sub(' +',' ',txtstr) # extra white spaces
    txtstr = txtstr.lower()
    return txtstr

# remove stopwords
def remove_stopwords(word_tokens): # must be tokenized sentence
    stop_words = set(stopwords.words('english')) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    return filtered_sentence

# stem long string - must be tokenized
def stemstr(txtstr):
    ps = PorterStemmer()
    newstr = [ps.stem(curr_word) for curr_word in txtstr]
    return newstr

# for figuring out what type of word
def get_pos( word ):
    w_synsets = wordnet.synsets(word)

    pos_counts = Counter()
    pos_counts["n"] = len(  [ item for item in w_synsets if item.pos()=="n"]  )
    pos_counts["v"] = len(  [ item for item in w_synsets if item.pos()=="v"]  )
    pos_counts["a"] = len(  [ item for item in w_synsets if item.pos()=="a"]  )
    pos_counts["r"] = len(  [ item for item in w_synsets if item.pos()=="r"]  )
    
    most_common_pos_list = pos_counts.most_common(3)
    return most_common_pos_list[0][0] # first indexer for 
            # getting the top POS from list, second indexer for getting POS from tuple( POS: count )

# lemmatize long string - must be tokenized
def lemstr(txtstr):
    lz = WordNetLemmatizer()
    newstr = [lz.lemmatize(curr_word, get_pos(curr_word)) for curr_word in txtstr]
    return newstr

In [66]:
# add some basic features for each article
mongo_id = []
n_words = []
author_list = []
author_id_n = []
n_nouns = []
n_adj = []
n_verb = []
n_adv =[]
for curr_name in writer_df['website_name']:
    currcol = mydb[curr_name]
    y = currcol.find({"name": curr_name})
    # iterate through all articles for that author
    for curr_art in y:
        # mongo id number
        mongo_id.append(str(curr_art['_id']))
        # n words after cleaning and removing numbers and punctuation
        a = clean_text(curr_art['article'])
        b = word_tokenize(a)
        n_words.append(len(b))
        # author list
        author_list.append(curr_name)
        # author id n
        author_id_n.append(curr_art['writer_id'])
        # remove stopwords from story
        nostopwords = remove_stopwords(b)
        typesofspeech = [get_pos(i) for i in nostopwords]
        n_nouns.append(typesofspeech.count('n'))
        n_adj.append(typesofspeech.count('a'))
        n_verb.append(typesofspeech.count('v'))
        n_adv.append(typesofspeech.count('r'))
        
        
        
        

In [86]:
# build a pd of what we have so far
data = {'n_words': n_words, 'author_id_n': author_id_n, 'author_list': author_list, 'mongo_id': mongo_id,
        'n_nouns': n_nouns, 'n_adj': n_adj, 'n_verb': n_verb, 'n_adv': n_adv}
writer_feature_df = pd.DataFrame.from_dict(data)
writer_feature_df.head(50)

,n_words,author_id_n,author_list,mongo_id,n_nouns,n_adj,n_verb,n_adv
0,3033,693716,zachlowe_nba,5b9c705c11919a2b1a15c30d,1229,72,435,106
1,3033,693716,zachlowe_nba,5b9c705d11919a2b1a15c30e,1229,72,435,106
2,2744,693716,zachlowe_nba,5b9c705f11919a2b1a15c30f,971,64,383,120
3,2886,693716,zachlowe_nba,5b9c706011919a2b1a15c310,1028,60,444,94
4,2886,693716,zachlowe_nba,5b9c706111919a2b1a15c311,1028,60,444,94
5,2555,693716,zachlowe_nba,5b9c706311919a2b1a15c312,784,70,410,65
6,2243,693716,zachlowe_nba,5b9c706411919a2b1a15c313,831,46,333,80
7,213,693716,zachlowe_nba,5b9c706611919a2b1a15c314,89,3,39,1
8,2215,693716,zachlowe_nba,5b9c706811919a2b1a15c315,820,46,327,79
9,4564,693716,zachlowe_nba,5b9c706911919a2b1a15c316,1765,114,659,155


In [87]:
# save writer feature df
with open('/Users/rohanramesh/Documents/Insight/data_bball_writers/writer_feature_df.pickle', 'wb') as handle:
    pickle.dump(writer_feature_df, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [81]:
# do analysis but by sentence
# curr_name = 'zachlowe_nba'
# currcol = mydb[curr_name]
# y = currcol.find({"name": curr_name})
# curr_art = y[5]

broken_sentences = sent_tokenize(curr_art['article'])
for i in broken_sentences:
    a = clean_text(i)

On Feb. 3 in Sacramento before an otherwise unremarkable Mavericks-Kings game, Jenny Boucek, a Sacramento Kings assistant coach, strode across the court toward Rick Carlisle, the Dallas Mavericks head coach, for what Carlisle assumed would be a typical pregame chat.
on feb in sacramento before an otherwise unremarkable mavericks kings game jenny boucek a sacramento kings assistant coach strode across the court toward rick carlisle the dallas mavericks head coach for what carlisle assumed would be a typical pregame chat 
The two are friends; Boucek, an inaugural WNBA player and longtime assistant and head coach in that league, had spent parts of the 2011 and 2014 WNBA offseasons visiting the Mavericks -- including for a full month of the team's 2014 training camp.
the two are friends boucek an inaugural wnba player and longtime assistant and head coach in that league had spent parts of the and wnba offseasons visiting the mavericks including for a full month of the teams training camp 


In [8]:
article_df = y[2]
article_df['article']

'Jimmy Butler in June 2017 fell right into the nebulous region where debating a maximum or new supermaximum contract becomes especially fraught for teams.\n\nWith another All-NBA berth the next season (2017-18), Butler would qualify for a five-year, $220 million supermax -- vaulting to a salary equivalent to 35 percent of the salary cap, 5 percentage points more than normal for a player at his experience level. The difference would amount to $25 million or more over the full deal. Add annual 8 percent raises that might outpace growth in the cap, and such a deal could soak up 40 percent of a team\'s cap space on the back end. It is as large a commitment as exists in the NBA.\n\nButler was the only star on a mediocre team. Without finding a second, and perhaps a third, the Bulls would go nowhere interesting during the length of Butler\'s supermax. The sheer size of his deal could make luring those stars harder.\n\nButler was about to turn 28. Depending on your taste, he at that point was

In [17]:
a = clean_text(article_df['article'])
a

'jimmy butler in june fell right into the nebulous region where debating a maximum or new supermaximum contract becomes especially fraught for teams with another all nba berth the next season butler would qualify for a five year million supermax vaulting to a salary equivalent to percent of the salary cap percentage points more than normal for a player at his experience level the difference would amount to million or more over the full deal add annual percent raises that might outpace growth in the cap and such a deal could soak up percent of a teams cap space on the back end it is as large a commitment as exists in the nba butler was the only star on a mediocre team without finding a second and perhaps a third the bulls would go nowhere interesting during the length of butlers supermax the sheer size of his deal could make luring those stars harder butler was about to turn depending on your taste he at that point was somewhere between the eighth and best player in the league you do th

In [18]:
b = sent_tokenize(article_df['article'])
c = word_tokenize(a)
print((c))

['jimmy', 'butler', 'in', 'june', 'fell', 'right', 'into', 'the', 'nebulous', 'region', 'where', 'debating', 'a', 'maximum', 'or', 'new', 'supermaximum', 'contract', 'becomes', 'especially', 'fraught', 'for', 'teams', 'with', 'another', 'all', 'nba', 'berth', 'the', 'next', 'season', 'butler', 'would', 'qualify', 'for', 'a', 'five', 'year', 'million', 'supermax', 'vaulting', 'to', 'a', 'salary', 'equivalent', 'to', 'percent', 'of', 'the', 'salary', 'cap', 'percentage', 'points', 'more', 'than', 'normal', 'for', 'a', 'player', 'at', 'his', 'experience', 'level', 'the', 'difference', 'would', 'amount', 'to', 'million', 'or', 'more', 'over', 'the', 'full', 'deal', 'add', 'annual', 'percent', 'raises', 'that', 'might', 'outpace', 'growth', 'in', 'the', 'cap', 'and', 'such', 'a', 'deal', 'could', 'soak', 'up', 'percent', 'of', 'a', 'teams', 'cap', 'space', 'on', 'the', 'back', 'end', 'it', 'is', 'as', 'large', 'a', 'commitment', 'as', 'exists', 'in', 'the', 'nba', 'butler', 'was', 'the', 'o

In [55]:
d = word_tokenize(b[0])
print(d)
print(lemstr(['fell', 'fall', 'flew', 'flies']))

['Jimmy', 'Butler', 'in', 'June', '2017', 'fell', 'right', 'into', 'the', 'nebulous', 'region', 'where', 'debating', 'a', 'maximum', 'or', 'new', 'supermaximum', 'contract', 'becomes', 'especially', 'fraught', 'for', 'teams', '.']
['fell', 'fall', 'fly', 'fly']


In [56]:
clean_text(b[0])

'jimmy butler in june fell right into the nebulous region where debating a maximum or new supermaximum contract becomes especially fraught for teams '